GSE169159: blood, healthy afaik, longitudinal 
GSE171964: Single-cell data, deseased, PBMC

Preprocessing bulk: 
- outlier detection with PCA (3 samples removed as they were more than 1.5sd away)
raw counts available 
- RLE > 0.6 removed 
- normalised by DESeq2 / BTMsT8

Relative log expression (RLE) plots were generated with EDAseq40; samples with an RLE > 0.6 were removed from the analysis. Differential gene analysis was performed using DESeq241 (v.1.26.0), incorporating participant identifier into the model to account for inter-participant bias. Genes were ranked by the Wald statistic as reported by DESeq2 for GSEA using the BTMs18. Per-participant fold changes were computed by dividing the DESeq2 normalized expression data for the day of interest by either day 0 (for day 1, day2 and day 7) or day 21 (for day 22, 28 and 42). To obtain BTM correlates with age, the age of each participant was compared against the per-participant fold changes for day 22. The resulting correlation values were ranked by t-statistic and analysed with GSEA. The same method was employed to obtain BTM correlates with IFNγ. IFN scores were computed by taking the per-participant mean fold change on day 22 of the unique set of genes present in the 5 interferon BTMs (M75, M111.1, M150, M127 and M68) that significantly correlated with day-22 IFNγ fold change. Similarly, the per-participant M16 gene score was computed using average fold change on day 22 of the genes present in M16.


Check: GSE97590 -> normalized count matrix from GEO used... -> we see this paper also does not compare microarray data with RNseq
(Baseline normalized log2-transformed fold changes were then computed per subject for all genes. GSEA was then performed to identify enriched BTMs using gene lists for each dataset ranked by t-statistic from two-sided Student’s t-tests on the post-vaccination log2-transformed fold changes.)




In [21]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.io import mmread


In [22]:
#With his I mostly want to find out how i can integrate this papers data into ImmPort
# TODO find out what is _BL (i suspect Baseline): Check it means baseline, i gutesss i will take this for the GAMLSS

In [23]:
## Testwise integration of 2020_Arachandra_Wimmers
root_folder = '/var/datasets/single_papers/2021_Arunachalam_vaccine/'

metadata_df = pd.read_csv(f'{root_folder}/metadata.txt')
print(metadata_df.shape)
metadata_df.head()

(185, 29)


,Run,Age,Assay Type,AvgSpotLen,Bases,BioProject,BioSample,Bytes,Center Name,Consent,...,LibrarySource,Organism,Platform,ReleaseDate,create_date,version,Sample Name,sex,source_name,SRA Study
0,SRR18038247,30,RNA-Seq,101,3816732127,PRJNA715457,SAMN25966748,1203925664,GEO,public,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2022-02-25T00:00:00Z,2022-02-15T13:52:00Z,1,GSM5900628,female,blood,SRP311216
1,SRR18038248,30,RNA-Seq,101,4274587852,PRJNA715457,SAMN25966749,1351097515,GEO,public,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2022-02-25T00:00:00Z,2022-02-15T13:50:00Z,1,GSM5900629,female,blood,SRP311216
2,SRR18038249,30,RNA-Seq,101,4188481110,PRJNA715457,SAMN25966750,1314161957,GEO,public,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2022-02-25T00:00:00Z,2022-02-15T13:49:00Z,1,GSM5900630,female,blood,SRP311216
3,SRR18038250,30,RNA-Seq,101,4373580376,PRJNA715457,SAMN25966751,1370039906,GEO,public,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2022-02-25T00:00:00Z,2022-02-15T13:49:00Z,1,GSM5900631,female,blood,SRP311216
4,SRR18038251,30,RNA-Seq,101,3491091866,PRJNA715457,SAMN25966752,1095712135,GEO,public,...,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2022-02-25T00:00:00Z,2022-02-15T13:46:00Z,1,GSM5900632,female,blood,SRP311216


In [24]:
counts_df = pd.read_csv(f'{root_folder}/GSE169159_raw_counts_with_genenames.csv', index_col='gene')
print(counts_df.shape)
counts_df.head()

(23640, 185)


,PID2053_BL,PID2053_Day1.2,PID2053_Day7,PID2053_Day21,PID2053_Day22.23,PID2053_Day28,PID2028_BL,PID2028_Day1.2,PID2028_Day7,PID2028_Day21,...,PID2001_Day7,PID2001_Day21,PID2001_Day22.23,PID2001_Day28,PID2046_BL,PID2046_Day1.2,PID2046_Day7,PID2046_Day21,PID2046_Day22.23,PID2046_Day28
gene,,,,,,,,,,,,,,,,,,,,,
FGR,16312,20268,13433,15317,18685,19379,11275,22251,939,11231,...,33607,21610,37503,39622,30299,19013,56015,16061,23595,23312
STPG1,137,150,154,113,136,179,92,108,105,126,...,86,55,62,63,92,111,246,60,117,97
NIPAL3,2247,2084,1808,1692,1576,2349,1223,1245,735,1197,...,1404,844,1023,782,847,1162,1903,781,1241,1239
KDM1A,1148,1047,876,813,1049,1133,773,778,167,620,...,1270,676,605,695,1019,988,1748,698,1306,930
E2F2,739,705,1312,1290,1878,1111,1189,1254,282,763,...,1302,719,255,677,1150,1197,3270,1622,1226,2080


In [25]:

# Problem: SRA Toolkit metadata does not have the study Subject accessions ...  Let´s see how else i can get them 
import sys
import importlib
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
import geo_utils
from geo_utils import read_prefixes_from_metadata_txt
importlib.reload(geo_utils)

path_metadata_txt = '/var/datasets/single_papers/2021_Arunachalam_vaccine/GSE169159_series_matrix.txt' # This is not the SRA Run oolkit meta! Its the GEO metadata.txt
metadata_geo_df = read_prefixes_from_metadata_txt(path_metadata_txt) # here we get age and sex info but my method yet overwrite it with the last of that tag (here it was !Sample_characteristics_ch1)
print(metadata_geo_df.shape)
## As this meta file saves several infos under the same "tag" I manually need to rename the cols to get desired info


metadata_geo_df = metadata_geo_df.rename(columns={'characteristics_ch1':'age', 
                                'characteristics_ch1.2':'day', 
                                'characteristics_ch1.3': 'sex'})
metadata_geo_df['age'] = metadata_geo_df['age'].str.lstrip('age: ')
metadata_geo_df['day'] = metadata_geo_df['day'].str.lstrip('day: ')
metadata_geo_df['sex'] = metadata_geo_df['sex'].str.lstrip('Sex: ')

metadata_geo_df.head()

(185, 37)


/home/eliasschreiner/PROJEKT/utils/geo_utils.py:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)


,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,age,...,contact_zip/postal_code,contact_country,data_row_count,instrument_model,library_selection,library_source,library_strategy,relation,relation.2,supplementary_file_1
0,PID2053_BL,GSM5900628,Public on Feb 16 2022,Feb 15 2022,Feb 16 2022,SRA,1,blood,Homo sapiens,30,...,94305,USA,0,Illumina MiSeq,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5900...
1,PID2053_Day1.2,GSM5900629,Public on Feb 16 2022,Feb 15 2022,Feb 16 2022,SRA,1,blood,Homo sapiens,30,...,94305,USA,0,Illumina MiSeq,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5900...
2,PID2053_Day7,GSM5900630,Public on Feb 16 2022,Feb 15 2022,Feb 16 2022,SRA,1,blood,Homo sapiens,30,...,94305,USA,0,Illumina MiSeq,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5900...
3,PID2053_Day21,GSM5900631,Public on Feb 16 2022,Feb 15 2022,Feb 16 2022,SRA,1,blood,Homo sapiens,30,...,94305,USA,0,Illumina MiSeq,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5900...
4,PID2053_Day22.23,GSM5900632,Public on Feb 16 2022,Feb 15 2022,Feb 16 2022,SRA,1,blood,Homo sapiens,30,...,94305,USA,0,Illumina MiSeq,cDNA,transcriptomic,RNA-Seq,BioSample: https://www.ncbi.nlm.nih.gov/biosam...,SRA: https://www.ncbi.nlm.nih.gov/sra?term=SRX...,ftp://ftp.ncbi.nlm.nih.gov/geo/samples/GSM5900...


In [26]:
counts_df_T = counts_df.T
combi_df = counts_df.merge(metadata_geo_df, right_on='title', how='left', left_on='gene')
print(combi_df.shape)
combi_df.head()

(23640, 222)


,PID2053_BL,PID2053_Day1.2,PID2053_Day7,PID2053_Day21,PID2053_Day22.23,PID2053_Day28,PID2028_BL,PID2028_Day1.2,PID2028_Day7,PID2028_Day21,...,contact_zip/postal_code,contact_country,data_row_count,instrument_model,library_selection,library_source,library_strategy,relation,relation.2,supplementary_file_1
0,16312,20268,13433,15317,18685,19379,11275,22251,939,11231,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,137,150,154,113,136,179,92,108,105,126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2247,2084,1808,1692,1576,2349,1223,1245,735,1197,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1148,1047,876,813,1049,1133,773,778,167,620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,739,705,1312,1290,1878,1111,1189,1254,282,763,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
combi_df.to_csv('2021_aranachalam_vaccine_combi_df.csv')

In [28]:
counts_df_two_filtered_T = counts_df_two_filtered.T
counts_df_two_filtered_T

gene,FGR,STPG1,NIPAL3,KDM1A,E2F2,CDK11A,PAX7,MASP2,ELOA,LYPLA2,...,EIF3FP1,SIK1B,KCNE1B,CYCSP41,CDRT15P9,SOWAHCP2,NCOR1P4,GATD3B,PLAC4,PCSEAT
PID2053_BL,16312,137,2247,1148,739,60,0,21,954,3368,...,0,7,7,0,0,0,0,80,38,0
PID2028_BL,11275,92,1223,773,1189,48,18,22,696,2438,...,17,4,6,0,1,0,0,24,28,5
PID2013_BL,11526,209,1707,997,503,52,5,14,990,2470,...,18,15,3,0,0,0,4,127,24,0
PID2045_BL,17423,136,1427,670,1520,19,2,9,787,3750,...,1,10,8,0,0,0,0,82,15,0
PID2018_BL,17222,132,1377,595,2667,46,3,15,933,3177,...,0,7,7,0,1,2,0,99,22,2
PID2042_BL,16577,98,902,520,2098,38,0,15,639,2074,...,0,1,4,0,0,0,0,35,6,0
PID2007_BL,38196,128,1535,1058,1394,77,1,10,1172,4521,...,0,3,2,0,0,0,1,100,13,0
PID2027_BL,19233,145,1350,733,1053,58,14,15,787,3588,...,13,6,2,0,1,0,0,78,39,1
PID2044_BL,16323,159,2028,1088,665,50,28,33,1165,2992,...,6,2,3,1,2,0,1,99,71,4
PID2048_BL,19316,95,1302,576,2858,43,0,11,641,2835,...,0,4,3,0,0,0,0,5,7,0


In [35]:
## Now I want to have only the Baseline data, as we dont need all the longituninal stuff and also thats not a healthy anymore
counts_df_two = counts_df
counts_df_two_filtered = counts_df_two.filter(regex='_BL$')

counts_df_two_filtered_T = counts_df_two_filtered.T
combi_df_baselines = counts_df_two_filtered_T.merge(metadata_geo_df, right_on='title', how='left', left_index=True)
meta_cols = list(combi_df_baselines.columns[-37:])
count_cols = list(combi_df_baselines.columns[:-37])
new_order_cols = meta_cols + count_cols
combi_df_baselines = combi_df_baselines[new_order_cols]
print(combi_df_baselines.shape)
combi_df_baselines.head()

(32, 23677)


,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,age,...,EIF3FP1,SIK1B,KCNE1B,CYCSP41,CDRT15P9,SOWAHCP2,NCOR1P4,GATD3B,PLAC4,PCSEAT
0,PID2053_BL,GSM5900628,Public on Feb 16 2022,Feb 15 2022,Feb 16 2022,SRA,1,blood,Homo sapiens,30,...,0,7,7,0,0,0,0,80,38,0
6,PID2028_BL,GSM5900634,Public on Feb 16 2022,Feb 15 2022,Feb 16 2022,SRA,1,blood,Homo sapiens,35,...,17,4,6,0,1,0,0,24,28,5
12,PID2013_BL,GSM5900640,Public on Feb 16 2022,Feb 15 2022,Feb 16 2022,SRA,1,blood,Homo sapiens,55,...,18,15,3,0,0,0,4,127,24,0
17,PID2045_BL,GSM5900645,Public on Feb 16 2022,Feb 15 2022,Feb 16 2022,SRA,1,blood,Homo sapiens,74,...,1,10,8,0,0,0,0,82,15,0
23,PID2018_BL,GSM5900651,Public on Feb 16 2022,Feb 15 2022,Feb 16 2022,SRA,1,blood,Homo sapiens,28,...,0,7,7,0,1,2,0,99,22,2


In [36]:
combi_df_baselines.to_csv('2021_aranachalam_vaccine_combi_df_baseline.csv')